In [6]:
import pandas as pd
import numpy as np
from catboost import Pool, CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit 
import optuna 
import json
from sklearn.metrics import mean_absolute_error

In [7]:
#Load inn datasets
X_test  = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_test.parquet')
X_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/X_train.parquet')
y_train = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train.parquet')
y_train_a = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_a.parquet')
y_train_b = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_b.parquet')
y_train_c = pd.read_parquet('../data/prepared_datasets/only_y_cleaned/Y_train_c.parquet')

In [8]:
def splitting_def(df):
    date_range_1 = (df.index >= '2020-05-01') & (df.index <= '2020-06-25')
    date_range_2 = (df.index >= '2023-05-01') & (df.index <= '2023-06-15')

    # Combine the date ranges to create the test set
    test_set = df[date_range_1 | date_range_2]

    # The rest of the data will be your training set
    training_set = df[~(date_range_1 | date_range_2)]
    
    # Splitting the test_set into X_test and y_test
    X_test = test_set.drop("pv_measurement", axis=1)
    y_test = test_set['pv_measurement']  # Assuming 'pv_measurement' is your target variable

    # Splitting the training_set into X_train and y_train
    X_train = training_set.drop("pv_measurement", axis=1)
    y_train = training_set['pv_measurement']
    
    return X_train, X_test, y_train, y_test

X_train_new_a, X_test_new_a, y_train_new_a, y_test_a = splitting_def(pd.concat([X_train[X_train["location"] == "A"].drop("location", axis=1), y_train_a], axis=1))
X_train_new_b, X_test_new_b, y_train_new_b, y_test_b = splitting_def(pd.concat([X_train[X_train["location"] == "B"].drop("location", axis=1), y_train_b], axis=1))
X_train_new_c, X_test_new_c, y_train_new_c, y_test_c = splitting_def(pd.concat([X_train[X_train["location"] == "C"].drop("location", axis=1), y_train_c], axis=1))

In [9]:
#Create a pool of data
train_pool_a = Pool(X_train_new_a, y_train_new_a)
train_pool_b = Pool(X_train_new_b, y_train_new_b)
train_pool_c = Pool(X_train_new_c, y_train_new_c)


test_pool_a = Pool(X_test_new_a) 
test_pool_b = Pool(X_test_new_b) 
test_pool_c = Pool(X_test_new_c) 

In [14]:
#For location A
def objective(trial, X_train, y_train):
    params = {
        "iterations": trial.suggest_int("iterations", 300, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 13),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
        "l2_leaf_reg": trial.suggest_int("l2_leaf_reg", 2, 10),
        "has_time": trial.suggest_categorical('has-time', [True, False]),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.3, 1.0),
        "random_strength": trial.suggest_float("random_strength", 0.3, 1.0),
        "random_state": 2,
        "border_count": trial.suggest_int("border_count", 1, 1000),
        "rsm": trial.suggest_float("rsm", 0.05, 1),
        "nan_mode": trial.suggest_categorical("nan_mode", ["Min", "Max"])
    }

    catboost_model_a = CatBoostRegressor(**params, verbose=100)
    catboost_model_a.fit(train_pool_a)
    pred_a = pd.DataFrame(catboost_model_a.predict(test_pool_a))
    MAE_a = mean_absolute_error(y_test_a, pred_a)
    return MAE_a

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=60)

[I 2023-11-06 20:06:33,040] A new study created in memory with name: no-name-39b145e3-dffd-4b30-bf12-f497d6a660d6


0:	learn: 1130.3922126	total: 35.8ms	remaining: 1m 8s
100:	learn: 601.4970272	total: 2.77s	remaining: 50s
200:	learn: 465.2164365	total: 5.38s	remaining: 46.1s
300:	learn: 427.7664397	total: 7.99s	remaining: 43.1s
400:	learn: 412.0486311	total: 10.6s	remaining: 40.4s
500:	learn: 401.5970133	total: 13.2s	remaining: 37.5s
600:	learn: 392.3700976	total: 15.7s	remaining: 34.6s
700:	learn: 384.5275376	total: 18.3s	remaining: 32s
800:	learn: 377.1771802	total: 20.9s	remaining: 29.4s
900:	learn: 369.8250113	total: 24.2s	remaining: 27.5s
1000:	learn: 362.3333322	total: 27.9s	remaining: 25.7s
1100:	learn: 355.8708999	total: 32.2s	remaining: 24.1s
1200:	learn: 349.3497994	total: 38.4s	remaining: 23.1s
1300:	learn: 343.3554911	total: 44.6s	remaining: 21.4s
1400:	learn: 337.4736538	total: 50.9s	remaining: 19s
1500:	learn: 331.7856860	total: 58s	remaining: 16.3s
1600:	learn: 326.1604720	total: 1m 5s	remaining: 13.2s
1700:	learn: 320.9741494	total: 1m 12s	remaining: 9.56s
1800:	learn: 315.8109170	to

[I 2023-11-06 20:08:03,588] Trial 0 finished with value: 336.212519318066 and parameters: {'iterations': 1924, 'learning_rate': 0.010531560739043, 'depth': 10, 'min_data_in_leaf': 73, 'l2_leaf_reg': 7, 'has-time': False, 'bagging_temperature': 0.4654000551729968, 'random_strength': 0.9912286171083025, 'border_count': 641, 'rsm': 0.33678356404760373, 'nan_mode': 'Min'}. Best is trial 0 with value: 336.212519318066.


0:	learn: 1081.9915457	total: 211ms	remaining: 8m 25s
100:	learn: 394.4222958	total: 13.4s	remaining: 5m 4s
200:	learn: 351.1494378	total: 26.5s	remaining: 4m 49s
300:	learn: 318.8973705	total: 39.9s	remaining: 4m 37s
400:	learn: 295.0073237	total: 53s	remaining: 4m 23s
500:	learn: 273.7891367	total: 1m 6s	remaining: 4m 10s
600:	learn: 254.8779881	total: 1m 17s	remaining: 3m 52s
700:	learn: 237.6732527	total: 1m 29s	remaining: 3m 35s
800:	learn: 223.8673829	total: 1m 41s	remaining: 3m 22s
900:	learn: 210.8339872	total: 1m 53s	remaining: 3m 8s
1000:	learn: 200.3856612	total: 2m 8s	remaining: 2m 58s
1100:	learn: 189.3191043	total: 2m 21s	remaining: 2m 46s
1200:	learn: 180.1645808	total: 2m 35s	remaining: 2m 34s
1300:	learn: 170.9256777	total: 2m 49s	remaining: 2m 22s
1400:	learn: 162.4172305	total: 3m 4s	remaining: 2m 11s
1500:	learn: 154.7574815	total: 3m 18s	remaining: 1m 58s
1600:	learn: 147.5962550	total: 3m 31s	remaining: 1m 45s
1700:	learn: 140.8317124	total: 3m 45s	remaining: 1m 3

[I 2023-11-06 20:13:25,761] Trial 1 finished with value: 343.2254001969638 and parameters: {'iterations': 2397, 'learning_rate': 0.0648118716151209, 'depth': 9, 'min_data_in_leaf': 41, 'l2_leaf_reg': 4, 'has-time': True, 'bagging_temperature': 0.3870705308963034, 'random_strength': 0.792704501532407, 'border_count': 957, 'rsm': 0.8647522929198236, 'nan_mode': 'Min'}. Best is trial 0 with value: 336.212519318066.


...

0:	learn: 1121.8935433	total: 298ms	remaining: 14m 46s
100:	learn: 455.7200704	total: 1m 19s	remaining: 37m 47s
200:	learn: 386.3221832	total: 2m 39s	remaining: 36m 39s
300:	learn: 354.9694261	total: 3m 59s	remaining: 35m 27s
400:	learn: 331.3349753	total: 5m 19s	remaining: 34m 8s
500:	learn: 311.7931529	total: 6m 36s	remaining: 32m 40s
600:	learn: 295.7145478	total: 7m 55s	remaining: 31m 20s
700:	learn: 279.4972688	total: 9m 14s	remaining: 29m 59s
800:	learn: 265.3246505	total: 10m 28s	remaining: 28m 26s
900:	learn: 254.2747894	total: 11m 38s	remaining: 26m 48s
1000:	learn: 243.5941281	total: 12m 51s	remaining: 25m 21s
1100:	learn: 233.7704632	total: 14m 2s	remaining: 23m 54s
1200:	learn: 224.5627271	total: 15m 10s	remaining: 22m 25s
1300:	learn: 215.9976016	total: 16m 13s	remaining: 20m 53s
1400:	learn: 208.4485330	total: 17m 14s	remaining: 19m 22s
1500:	learn: 201.9416459	total: 18m 14s	remaining: 17m 55s
1600:	learn: 194.9279494	total: 19m 19s	remaining: 16m 35s
1700:	learn: 186.92

[I 2023-11-06 21:27:28,744] Trial 19 finished with value: 336.0104191154252 and parameters: {'iterations': 2976, 'learning_rate': 0.02123102183183657, 'depth': 13, 'min_data_in_leaf': 86, 'l2_leaf_reg': 9, 'has-time': False, 'bagging_temperature': 0.7188175228917684, 'random_strength': 0.8916564476921582, 'border_count': 814, 'rsm': 0.5676326068210172, 'nan_mode': 'Max'}. Best is trial 18 with value: 333.5201363181928.


0:	learn: 1102.1593622	total: 406ms	remaining: 20m 11s
100:	learn: 382.6366157	total: 50.1s	remaining: 23m 51s


[W 2023-11-06 21:28:36,150] Trial 20 failed with parameters: {'iterations': 2989, 'learning_rate': 0.04308761823821933, 'depth': 13, 'min_data_in_leaf': 87, 'l2_leaf_reg': 9, 'has-time': True, 'bagging_temperature': 0.7815230039161076, 'random_strength': 0.816693500602857, 'border_count': 832, 'rsm': 0.5729611011120581, 'nan_mode': 'Max'} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/tmp/ipykernel_2035/311075365.py", line 25, in <lambda>
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=60)
  File "/var/tmp/ipykernel_2035/311075365.py", line 19, in objective
    catboost_model_a.fit(train_pool_a)
  File "/opt/conda/lib/python3.10/site-packages/catboost/core.py", line 5703, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_wei

KeyboardInterrupt: 

In [10]:
#to output the best paramaters
print(study.best_params)

#to output the best score returned from the trials
print(study.best_value)


with open("optuna-best-parameters_a.txt", "w") as file:
    file.write("Best paramaters: \n")
    file.write(json.dumps(study.best_params))  # Write the first string followed by a newline character
    file.write("\n")
    file.write("best score MAE: \n")
    file.write(json.dumps(study.best_value))  # Write the second string followed by a newline character

{'iterations': 1852, 'learning_rate': 0.0018854412552105353, 'depth': 9, 'colsample_bylevel': 0.7366562974962199, 'min_data_in_leaf': 4, 'l2_leaf_reg': 3, 'has-time': True, 'bagging_temperature': 0.9525931510378696, 'random_strength': 0.30613129187014626, 'border_count': 993, 'rsm': 0.21119141087462187, 'nan_mode': 'min'}
351.34628571238125
